<a href="https://colab.research.google.com/github/shilpasy/Data-manipulation-and-visualization-of-gene-expression-data-from-NCBI-GEO/blob/main/DW_rancho.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
import numpy as np

In [ ]:
excel_file_path = '/content/drive/My Drive/kaggle_data/DW_rancho/Technical Test - Data Wrangling.xlsx'

# Reading the given Excel file into a dictionary of DataFrames
dfs_from_excel = pd.read_excel(excel_file_path, sheet_name=None)

# Access individual sheets by specifying their names
rnaseq = dfs_from_excel['RNA-seq (RPKM)']
tissue = dfs_from_excel['Tissue Sample Metadata']
prot_meta = dfs_from_excel['Serum Protein data']
patient = dfs_from_excel['Patient_clinical_data']

tissue = tissue.rename(columns={'Patient  Number': 'Patient_ID', 'Sample': 'Sample_ID'})
tissue['Patient_ID'] = tissue['Patient_ID'].astype(int)
prot_meta = prot_meta.rename(columns={'Patient': 'Patient_ID', 'Sample': 'Sample_ID'})
prot_meta['Patient_ID'] = prot_meta['Patient_ID'].astype(int)

patient = patient.rename(columns={'Patient  Number': 'Patient_ID'})
patient['Patient_ID'] = patient['Patient_ID'].astype(int)
patient['Age'] = patient['Age'].astype(int)

In [ ]:
df = patient.merge(prot_meta, on='Patient_ID', how='outer', suffixes=('_left', '_right'))
df['Material_type'] = 'SERUM'
df.head(2)

,Study_ID,Patient_ID,Sex,Age,Sample_ID,Serum IL-6 (g/L),Serum IL-6 Receptor (mg/L),Material_type
0,RO4532A,1001,M,41,S-CRC1-A,73.7,0.97,SERUM
1,RO4532A,1001,M,41,S-CRC1-B,67.7,1.49,SERUM


In [ ]:
#df[['Serum IL-6 (g/L)', 'Serum IL-6 Receptor (mg/L)']] = df[['Serum IL-6 (g/L)', 'Serum IL-6 Receptor (mg/L)']].fillna('')

# Melt the DataFrame to create a 'gene symbol' column
merged_df1 = pd.melt(df, id_vars=['Study_ID', 'Patient_ID', 'Sex', 'Age', 'Sample_ID', 'Material_type'],
                    value_vars=['Serum IL-6 (g/L)', 'Serum IL-6 Receptor (mg/L)'], var_name='Gene_Symbol', value_name='Result')
# Map the gene symbols as desired
gene_symbol_mapping = {'Serum IL-6 (g/L)': 'IL6', 'Serum IL-6 Receptor (mg/L)': 'IL6R'}
merged_df1['Gene_Symbol'] = merged_df1['Gene_Symbol'].map(gene_symbol_mapping)
merged_df1['Result_Units'] = "g/L"
merged_df1['Sample_General_Pathology'] = "NA"
#In the cases where we do not have valid result, use "Not Done"
merged_df1['Status'] = merged_df1['Result'].apply(lambda x: 'NOT DONE' if not str(x).replace('.', '', 1).isnumeric() else "NA") #np.nan)

#merged_df1['Result'] = merged_df1.apply(lambda row: row['Result'] / 1000 if row['Gene_Symbol'] == 'IL6R' else row['Result'], axis = 1) # this should directly work but because there are non-numeric values in this column this isn't working

merged_df1['Result'] = pd.to_numeric(merged_df1['Result'], errors='coerce')

# Convert mg/L to g/L for uniformity, perform this for rows where 'Gene_Symbol' is 'IL6R'
merged_df1['Result'] = np.where(
    (merged_df1['Gene_Symbol'] == 'IL6R') & pd.notna(merged_df1['Result']),
    merged_df1['Result'] / 1000,
    merged_df1['Result']
)

merged_df1.head(2)

,Study_ID,Patient_ID,Sex,Age,Sample_ID,Material_type,Gene_Symbol,Result,Result_Units,Sample_General_Pathology,Status
0,RO4532A,1001,M,41,S-CRC1-A,SERUM,IL6,73.7,g/L,NA,NA
1,RO4532A,1001,M,41,S-CRC1-B,SERUM,IL6,67.7,g/L,NA,NA


In [ ]:
df2 = patient.merge(tissue, on='Patient_ID', how='outer', suffixes=('_left', '_right'))

In [ ]:
df2.drop(columns= [ 'RIN', 'Total Reads(millions)'], inplace = True)

In [ ]:
df2 = df2.rename(columns = {'Material' : 'Material_type'} )
df2.head(2)

,Study_ID,Patient_ID,Sex,Age,Sample_ID,Sample type,Material_type
0,RO4532A,1001,M,41,CRC1N,Normal,RNA
1,RO4532A,1001,M,41,CRC1T,Liver Tumor,RNA


In [ ]:
rnaseq_unpivoted = pd.melt(rnaseq, id_vars=['GeneID'], var_name='Sample_ID', value_name='Result')
rnaseq_unpivoted.head(2)

,GeneID,Sample_ID,Result
0,ICAM1,CRC1N,4.662774
1,IL6,CRC1N,0.000000


In [ ]:
merged_df2 = df2.merge(rnaseq_unpivoted, on='Sample_ID', how='outer') #suffixes=('_left', '_right')
merged_df2 = merged_df2.rename(columns = {'GeneID' : 'Gene_Symbol', 'Sample type': 'Sample_General_Pathology'} )
merged_df2['Status'] = merged_df2['Result'].apply(lambda x: 'NOT DONE' if pd.isna(x) or not str(x).replace('.', '', 1).isnumeric() else 'NA')
merged_df2['Result_Units'] = 'RPKM'
merged_df2.head()

,Study_ID,Patient_ID,Sex,Age,Sample_ID,Sample_General_Pathology,Material_type,Gene_Symbol,Result,Status,Result_Units
0,RO4532A,1001,M,41,CRC1N,Normal,RNA,ICAM1,4.662774,NA,RPKM
1,RO4532A,1001,M,41,CRC1N,Normal,RNA,IL6,0.000000,NA,RPKM
2,RO4532A,1001,M,41,CRC1N,Normal,RNA,IL6R,12.153740,NA,RPKM
3,RO4532A,1001,M,41,CRC1N,Normal,RNA,VCAM1,4.596351,NA,RPKM
4,RO4532A,1001,M,41,CRC1N,Normal,RNA,SELE,0.077086,NA,RPKM


In [ ]:
desired_columns = ["Study_ID", "Patient_ID", "Sex", "Age", "Sample_ID", "Sample_General_Pathology", "Material_type", "Gene_Symbol", "Result", "Result_Units", "Status"]

example_report = pd.concat([merged_df2[desired_columns], merged_df1[desired_columns]], axis=0)
example_report['Sex'] = example_report['Sex'].apply(lambda x: 'Male' if x == 'M' else ('Female' if x == 'F' else x))
example_report['Unique_Patient_ID'] = example_report['Study_ID'] + '_' + str(example_report['Patient_ID'])
example_report['Unique_Patient_ID'] = example_report.apply(lambda row: f"{row['Study_ID']}_{row['Patient_ID']}", axis=1)
example_report.insert(2, 'Unique_Patient_ID', example_report.pop('Unique_Patient_ID'))
example_report['Sex'] = example_report['Sex'].str.upper()

In [ ]:
example_report.to_csv('/content/drive/My Drive/kaggle_data/DW_rancho/example_report.csv')

In [ ]:
example_report

,Study_ID,Patient_ID,Unique_Patient_ID,Sex,Age,Sample_ID,Sample_General_Pathology,Material_type,Gene_Symbol,Result,Result_Units,Status
0,RO4532A,1001,RO4532A_1001,MALE,41,CRC1N,Normal,RNA,ICAM1,4.662774,RPKM,NA
1,RO4532A,1001,RO4532A_1001,MALE,41,CRC1N,Normal,RNA,IL6,0.000000,RPKM,NA
2,RO4532A,1001,RO4532A_1001,MALE,41,CRC1N,Normal,RNA,IL6R,12.153740,RPKM,NA
3,RO4532A,1001,RO4532A_1001,MALE,41,CRC1N,Normal,RNA,VCAM1,4.596351,RPKM,NA
4,RO4532A,1001,RO4532A_1001,MALE,41,CRC1N,Normal,RNA,SELE,0.077086,RPKM,NA
...,...,...,...,...,...,...,...,...,...,...,...,...
43,RO4532A,2310,RO4532A_2310,MALE,62,S-CRC10-b,NA,SERUM,IL6R,0.002070,g/L,NA
44,RO4532A,2310,RO4532A_2310,MALE,62,S-CRC10-c,NA,SERUM,IL6R,0.002480,g/L,NA
45,RO4532A,2311,RO4532A_2311,MALE,75,S-CRC11-a,NA,SERUM,IL6R,0.003450,g/L,NA
46,RO4532A,2311,RO4532A_2311,MALE,75,S-CRC11-b,NA,SERUM,IL6R,0.003010,g/L,NA


**Optional extra processing of the Data**

If we have any guesses for the S-CRC1-A, B, C we can add that information instead of missing on that. The A, B, C could also be Normal, Liver Tumor,Metastatic Lung but in that case D would be an outlier or something else. Or we can simple retain A,B,C,D and consult with experimentalist.

In [ ]:
example_report['Sample_ID'] = example_report['Sample_ID'].str.upper()

# conditions
conditions = [
    example_report['Sample_ID'].str.match(r'S-CRC\d+-A', case=False),
    example_report['Sample_ID'].str.match(r'S-CRC\d+-B', case=False),
    example_report['Sample_ID'].str.match(r'S-CRC\d+-C', case=False)
]

values = ['A', 'B', 'C']
#values = ['Normal', 'Liver Tumor', 'Metastatic Lung']

example_report['Sample_General_Pathology'] = np.select(conditions, values, example_report['Sample_General_Pathology'])